In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

In [3]:
cap = cv2.VideoCapture(0)
movement_data = []
total_movement_session = 0.0

prev_landmarks = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    results = hands.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = np.array([(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])

            if prev_landmarks is not None:
                movement = np.linalg.norm(landmarks - prev_landmarks, axis=1)
                total_movement = np.sum(movement)
                total_movement_session += total_movement
                timestamp = datetime.now().strftime("%H:%M:%S")

                movement_data.append((timestamp, total_movement))
                print(f"Hand movement at {timestamp}: {total_movement:.2f}")

            prev_landmarks = landmarks

    cv2.imshow('Hand Movement', frame)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
hands.close()

c:\Users\manas\miniconda3\envs\gpu-env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Hand movement at 10:39:13: 0.69
Hand movement at 10:39:13: 9.78
Hand movement at 10:39:13: 9.60
Hand movement at 10:39:13: 9.92
Hand movement at 10:39:13: 9.20
Hand movement at 10:39:13: 9.10
Hand movement at 10:39:13: 8.97
Hand movement at 10:39:13: 8.83
Hand movement at 10:39:13: 8.70
Hand movement at 10:39:13: 8.43
Hand movement at 10:39:13: 8.00
Hand movement at 10:39:13: 8.02
Hand movement at 10:39:13: 7.94
Hand movement at 10:39:13: 7.91
Hand movement at 10:39:13: 7.90
Hand movement at 10:39:13: 7.82
Hand movement at 10:39:13: 7.50
Hand movement at 10:39:13: 7.41
Hand movement at 10:39:13: 7.37
Hand movement at 10:39:13: 7.29
Hand movement at 10:39:13: 7.27
Hand movement at 10:39:14: 7.19
Hand movement at 10:39:14: 7.18
Hand movement at 10:39:14: 6.98
Hand movement at 10:39:14: 7.17
Hand movement at 10:39:14: 7.13
Hand movement at 10:39:14: 7.15
Hand movement at 10:39:14: 7.07
Hand movement at 10:39:14: 7.23
Hand movement at 10:39:14: 7.18
Hand movement at 10:39:14: 7.20
Hand mov

In [4]:
df = pd.DataFrame(movement_data, columns=['Timestamp', 'Movement'])
df['Percentage of Total Movement'] = (df['Movement'] / total_movement_session) * 100
df.to_csv('hand_movement_data.csv', index=False)
print(f"Total hand movement during session: {total_movement_session:.2f}")

Total hand movement during session: 8412.54
